In [ ]:
import pandas as pd
import os

In [ ]:
main_data_path = '/Users/jk1/stroke_datasets/ptiO2-Studie'
pbto2_meta_data_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/pbto2_for_extraction.csv'

In [ ]:
pbto2_meta_data_df = pd.read_csv(pbto2_meta_data_path)

In [ ]:
from utils import format_date_column

pbto2_meta_data_df['Recording Start Time'] = format_date_column(pbto2_meta_data_df['Recording Start Time'])

In [ ]:
pbto2_meta_data_df

In [ ]:
manually_verified_supplementary_data_files = {
    '23_03': ['P3', 'P4'],
    '23_04': ['P3'],
    '23_05': ['P3'],
    '23_06': ['P3', 'P4'],
    '23_09': ['P3'],
}

In [ ]:
cpp_df = pd.DataFrame()
ptio2_df = pd.DataFrame()
for year_dir in os.listdir(main_data_path):
    year_path = os.path.join(main_data_path, year_dir)
    if not os.path.isdir(year_path):
        continue
    for subj_dir in os.listdir(year_path):
        subj_path = os.path.join(year_path, subj_dir)
        if not os.path.isdir(subj_path):
            continue

        pbto2_files = []
        cpp_files = []
        for file in os.listdir(subj_path):
            file_path = os.path.join(subj_path, file)
            # check if a file starting with PbtO2 and not ending with quality.txt and not containing 'AvgTime' exists
            if file.startswith('PbtO2') and not file.endswith('quality.txt') and 'AvgTime' not in file:
                pbto2_files.append(file)
            if subj_dir in manually_verified_supplementary_data_files.keys():
                # if file startswith any of manually_verified_supplementary_data_files[subj_dir]
                if any([file.startswith(f) for f in manually_verified_supplementary_data_files[subj_dir]]) and not file.endswith('quality.txt') and 'AvgTime' not in file:
                    pbto2_files.append(file)
            if file.startswith('CPP') and not file.endswith('quality.txt') and 'AvgTime' not in file and 'CPP2' not in file:
                cpp_files.append(file)
                
        if len(pbto2_files) > 1:
            # remove files with 'inc' in the name
            pbto2_files = [f for f in pbto2_files if 'inc' not in f]
        
        subj_pbto2_df = pd.DataFrame()
        for pbto2_file in pbto2_files:
            pbto2_path = os.path.join(subj_path, pbto2_file)
            single_pbto2_df = pd.read_csv(pbto2_path, sep='\t')
            if 'inc' in pbto2_file:
                single_pbto2_df = single_pbto2_df[['Day#', 'ClockTime', 'PbtO2,na,Numeric,Float,LicoxLCX02 (mmHg):Med']]
            # set last column as to ptio2
            single_pbto2_df.columns = [*single_pbto2_df.columns[:-1], 'ptio2']
            subj_pbto2_df = pd.concat([subj_pbto2_df, single_pbto2_df])
            
                
        subj_cpp_df = pd.DataFrame()
        for cpp_file in cpp_files:
            cpp_path = os.path.join(subj_path, cpp_file)
            single_cpp_df = pd.read_csv(cpp_path, sep='\t')
            if 'inc' in cpp_file:
                single_cpp_df = single_cpp_df[['Day#', 'ClockTime', 'CPP,na,Numeric,Float,CARESCAPE (mmHg):Med']]
            single_cpp_df.columns = [*single_cpp_df.columns[:-1], 'cpp']
            subj_cpp_df = pd.concat([subj_cpp_df, single_cpp_df])
            
        subj_recording_start_date = pbto2_meta_data_df[pbto2_meta_data_df['pat_nr'] == subj_dir]['Recording Start Time']
        subj_recording_start_date = pd.to_datetime(subj_recording_start_date.str[:17], format="%Y %b %d %H:%M").dt.date.values[0]
            
        subj_pbto2_df['ClockTime'] = subj_pbto2_df['ClockTime'].apply(lambda x: x if len(x) > 5 else x + ':00')
        subj_cpp_df['ClockTime'] = subj_cpp_df['ClockTime'].apply(lambda x: x if len(x) > 5 else x + ':00')
        
        subj_pbto2_df['recording_start_date'] = subj_recording_start_date
        subj_pbto2_df['datetime'] = subj_pbto2_df['recording_start_date'] + pd.to_timedelta(subj_pbto2_df['Day#'], unit='d') 
        subj_pbto2_df['datetime'] = subj_pbto2_df['datetime'].astype(str) + ' ' + subj_pbto2_df['ClockTime']
        
        subj_cpp_df['recording_start_date'] = subj_recording_start_date
        subj_cpp_df['datetime'] = subj_cpp_df['recording_start_date'] + pd.to_timedelta(subj_cpp_df['Day#'], unit='d') 
        subj_cpp_df['datetime'] = subj_cpp_df['datetime'].astype(str) + ' ' + subj_cpp_df['ClockTime']
            
        subj_pbto2_df['pat_nr'] = subj_dir
        subj_cpp_df['pat_nr'] = subj_dir
        
        cpp_df = pd.concat([cpp_df, subj_cpp_df])
        ptio2_df = pd.concat([ptio2_df, subj_pbto2_df])
        
cpp_df = cpp_df.drop(columns=['MRN'])
ptio2_df = ptio2_df.drop(columns=['MRN'])           

In [ ]:
cpp_df.head()

In [ ]:
ptio2_df.head()

In [ ]:
cpp_df.to_csv('/Users/jk1/Downloads/cpp_df.csv', index=False)
ptio2_df.to_csv('/Users/jk1/Downloads/ptio2_df.csv', index=False)